In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [2]:
# # Define a simple model
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc = nn.Linear(10, 12)

#     def forward(self, x):
#         return self.fc(x)

# # Create an instance of the model and train it (dummy training here)
# model = SimpleModel()
# criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)

# # Dummy training loop
# for epoch in range(100):
#     inputs = torch.randn(5, 10)
#     targets = torch.randn(5, 12)
    
#     optimizer.zero_grad()
#     outputs = model(inputs)
#     loss = criterion(outputs, targets)
#     loss.backward()
#     optimizer.step()

# # Save the trained model
# torch.save(model.state_dict(), 'model.pth')

#### Inference

In [3]:
def visualize_debit(data, title, dates):
    # Convert date strings to datetime objects
    date_objects = [datetime.strptime(date, "%Y-%m-%d %H:%M") for date in dates]
    
    # Create a plot
    plt.figure(figsize=(8, 5))
    plt.plot(dates, data, marker='o', linestyle='-')  

    # Set font size for x-axis labels
    plt.gca().tick_params(axis='x', labelsize=6)  # Set font size here
    
    # Rotate date labels for better readability
    plt.gcf().autofmt_xdate()
    
    # Add title and labels
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('m3/hr')
    
    # Show plot
    plt.show()
    
def get_predicted_output(response):
    values = response['Output'][0]
    curr_date = response['Prediction Time']
    return values, curr_date

from datetime import datetime, timedelta

def generate_next_datetime(start_datetime, num_intervals):
    # Parse the initial datetime string into a datetime object
    start_datetime = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M")
    
    # List to hold all the datetime values
    datetime_list = []
    
    # Generate datetime values 1 hours apart
    for _ in range(0, num_intervals):
        start_datetime += timedelta(hours=1)
        datetime_list.append(start_datetime.strftime("%Y-%m-%d %H:%M"))
    
    return datetime_list

In [4]:
def run_prediction_debit():
    url = 'http://127.0.0.1:8000/predict'
    inputs = list(np.round(np.random.rand(10),3))
    # Example input data
    data = {
        'inputs': [inputs]
    }
    
    # Send a request to the server
    response = requests.post(url, json=data)
    
    #retrieve the response into json
    predicted_output = response.json()
    
    #get the value from the response
    prediction_value, prediction_time = get_predicted_output(predicted_output)
    predicted_time_sequence = generate_next_datetime(prediction_time, 12)
    
    #plot the predicted value
    title = f"Debit in the next {len(prediction_value)} hour start from {prediction_time}"
    visualize_debit(data=prediction_value, dates=predicted_time_sequence, title=title)
    clear_output(wait=True)  # Clear the output before the next plot

In [28]:
def run_ingest_data():
    url_get_input_data = 'http://127.0.0.1:8000/get_input_data'
    #send request to the server
    response = requests.post(url_get_input_data)

    #retrieve the response into json
    precip = response.json()
    
    precip_value = precip["Precip Value"]
    precip_value = np.array(precip_value)
    return precip_value

In [25]:
if __name__ == '__main__':
    precip_value = run_ingest_data()

In [27]:
from apscheduler.schedulers.blocking import BlockingScheduler

def main():  
    scheduler = BlockingScheduler()
    scheduler.add_job(run_prediction_debit, 'interval', seconds=3)
    
    scheduler.start()
    
if __name__ == '__main__':
    main()

Job "run_prediction_debit (trigger: interval[0:00:03], next run at: 2024-07-09 17:39:43 +07)" raised an exception
Traceback (most recent call last):
  File "C:\Users\62812\anaconda3\Lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\62812\AppData\Local\Temp\ipykernel_30456\248842236.py", line 16, in run_prediction_debit
    prediction_value, prediction_time = get_predicted_output(predicted_output)
                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\62812\AppData\Local\Temp\ipykernel_30456\1183070049.py", line 24, in get_predicted_output
    values = response['Output'][0]
             ~~~~~~~~^^^^^^^^^^
KeyError: 'Output'
Job "run_prediction_debit (trigger: interval[0:00:03], next run at: 2024-07-09 17:39:46 +07)" raised an exception
Traceback (most recent call last):
  File "C:\Users\62812\anaconda3\Lib\site-packa

KeyboardInterrupt: 